# EMNIST

In [1]:
%load_ext autoreload
%autoreload 2
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset

In [2]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)

cuda


In [3]:
# Create the working directory from timestamp and model name
model_name = 'cvae_emnist' # invertible network
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
workingDir = f'tb_logs/{model_name}_{timestamp}'
os.mkdir(workingDir)

In [4]:
lr = 1e-3
batch_size = 64
latent_dim = 4
TRAIN_CVAE = True

In [5]:
transform = transforms.Compose([transforms.ToTensor(),lambda x: x.rot90(1,[2,1]).flip(2)])#,  transforms.Resize((32,32))]) # Dataset transform #  lambda x: (x>0.5).float(),
dataset = datasets.EMNIST(root='/share/datasets', download=False, split='byclass', train=True, transform=transform)
new_dataset = partition_dataset(dataset, range(10, 36))
train_set, val_set = torch.utils.data.random_split(new_dataset, [round(0.8*len(new_dataset)), round(0.2*len(new_dataset))])

# Dataloaders

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=12, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=12, shuffle=False)

In [6]:
x,t=next(iter(train_loader))
torchvision.utils.save_image(x, 'image.pdf', nrow=8)
print(['abcdefghijklmnopqrstuvwxyz'[i] for i in t])

['h', 'f', 'm', 'x', 'm', 'c', 'c', 'e', 'a', 't', 'r', 'p', 'k', 'c', 'o', 'a', 'c', 'i', 'a', 'n', 'r', 'n', 'o', 'w', 'i', 't', 'o', 'c', 's', 'd', 'o', 'c', 'b', 'p', 'z', 'o', 'f', 'i', 'i', 'p', 'u', 'e', 'h', 's', 'x', 'm', 'z', 'a', 'o', 'd', 'm', 'w', 'h', 'u', 'o', 'd', 's', 't', 'h', 'w', 'y', 'r', 's', 's']


In [ ]:
tb_logger = TensorBoardLogger('tb_logs', name=model_name)
enc_layers = [28*28, 1024, 1024]
dec_layers = [1024, 1024, 28*28]
if TRAIN_CVAE:
    model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
    trainer = pl.Trainer(gpus=1,logger=tb_logger,max_epochs=100,auto_lr_find=True)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/nislah/.vmgr_repo/dev-2021-02-py38/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | EMNIST_Encoder | 1.9 M 
1 | decoder | EMNIST_Decoder | 1.9 M 
-------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.093    Total estimated model params size (MB)


Epoch 0:   1%| | 31/2953 [00:00<01:19, 36.80it/s, loss=287, v

/home/nislah/.vmgr_repo/dev-2021-02-py38/lib/python3.8/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'mu', 'targets'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


Epoch 0:  80%|▊| 2361/2953 [00:13<00:03, 178.29it/s, loss=175